In [1]:

import pandas as pd
# Load the CSV file into a pandas DataFrame
file_path = 'C:/Users/abhis/Desktop/CloudBuilders/Grafana-Anomaly/Grafana-Anomaly_Detection/Anomaly_grafana/notebooks/output_csv/duration_milliseconds_bucket.csv'  # Replace this with your file path 
df_ms_bucket= pd.read_csv(file_path)

# Display the first few rows to verify
df_ms_bucket.head()

#NAN VALUE TREATMENT------------------------------------------------------------------
# Display the number of NaN values in each column
df_ms_bucket.isna().sum()

#Interpolate
df_converted = df_ms_bucket.convert_dtypes()

# Select only numeric columns for interpolation
numeric_cols = df_converted.select_dtypes(include=['number']).columns

# Perform interpolation on numeric columns only
df_converted[numeric_cols] = df_converted[numeric_cols].interpolate()

# Check for any remaining NaN values
df_converted.isna().sum()

#Backward Fill
df_bfill = df_converted.bfill()
df_bfill.isna().sum()

#Forward Fill
df_ffill = df_bfill.ffill()
df_ffill.isna().sum()

#creating a final df
final_df = df_ffill
final_df.head()

#optional
#final_df.to_csv('C:/Users/abhis/Desktop/CloudBuilders/Grafana-Anomaly/Grafana-Anomaly_Detection/Anomaly_grafana/dataframes/duration_ms_bucket_df.csv', index=False)


# Check data types
#print(final_df.dtypes)

#summary statistics
#final_df.describe()



C:\Users\abhis\AppData\Local\Temp\ipykernel_4520\237286440.py:4: DtypeWarning: Columns (2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ms_bucket= pd.read_csv(file_path)
c:\Users\abhis\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:1057: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\abhis\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\dtypes\cast.py:1081: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [ ]:
#model with numerical and categorical anomalies (isolation_forest)

#DATA PROCESSING-----------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest

# Selecting the numerical and categorical columns
numerical_columns = ['Value', 'http_status_code']
categorical_columns = ['exported_instance', 'exported_job', 'http_method', 
                       'http_route', 'instance', 'job', 'service_name', 
                       'span_kind', 'span_name', 'status_code']

# Preprocessing: Scaling numerical data and encoding categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)])

# Creating the Isolation Forest model pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', IsolationForest(contamination=0.001, random_state=42))])

# Fit the model
model.fit(final_df)

# Predict anomalies (anomalies will be labeled as -1, normal as 1)
final_df['anomaly'] = model.fit_predict(final_df)

# Filter the anomalies
anomalies = final_df[final_df['anomaly'] == -1]

# Display the detected anomalies
anomalies

# PLOT THE ANOMALY
import matplotlib.pyplot as plt

# Convert 'Time' to datetime format
final_df['Time'] = pd.to_datetime(final_df['Time'])

# Plotting the time series data with anomalies
plt.figure(figsize=(14, 7))

# Plot all data points
plt.plot(final_df['Time'], final_df['Value'], label='Normal', color='blue', alpha=0.5)

# Highlight the anomalies
anomalies = final_df[final_df['anomaly'] == -1]
plt.scatter(anomalies['Time'], anomalies['Value'], color='red', label='Anomalies')

# Adding labels and title
plt.title('duration_milliseconds_bucket Isolation_Forest(num+cat)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()

# Show plot
plt.grid(True)
plt.show()
print(f"Number of anomalies detected: {len(anomalies)}")
print(anomalies.head())

In [ ]:
#model with only numerical anomalies (isolation_forest)

# Convert 'Time' column to datetime and set as index
final_df['Time'] = pd.to_datetime(final_df['Time'])
final_df.set_index('Time', inplace=True)
 
# Select all relevant numeric metric columns for anomaly detection
metrics_columns = ['Value']  # Add other metric columns as needed
metrics_data = final_df[metrics_columns]
 
# Standardize the data
scaler = StandardScaler()
metrics_scaled = scaler.fit_transform(metrics_data)
 
# Initialize the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, contamination=0.001, random_state=42)
 
# Fit the model and predict anomalies (-1 means anomaly, 1 means normal)
final_df['anomaly'] = iso_forest.fit_predict(metrics_scaled)
 
# Visualize the anomalies for each metric
for column in metrics_columns:
    plt.figure(figsize=(15, 6))
    plt.plot(final_df.index, final_df[column], label=column, color='blue')
    plt.scatter(final_df.index[final_df['anomaly'] == -1], final_df[column][final_df['anomaly'] == -1],
                color='red', label='Anomaly', marker='x')
    plt.title(f'duration_milliseconds_bucket Isolation_Forest(num)')
    plt.xlabel('Time')
    plt.ylabel(column)
    plt.legend()
    plt.show()
 
# Print out the anomalies
anomalies = final_df[final_df['anomaly'] == -1]
print(f"Number of anomalies detected: {len(anomalies)}")
print(anomalies.head())

In [ ]:
#Z-Score 
import pandas as pd
from scipy.stats import zscore

# Load your DataFrame (df) here
# df = pd.read_csv('your_file.csv')

# Compute z-scores for the "Value" column
final_df['Value_zscore'] = zscore(final_df['Value'])

# Set a threshold for anomaly detection (e.g., 3 standard deviations)
threshold =0.1

# Flag anomalies (True for anomalies, False otherwise)
final_df['Anomaly'] = (final_df['Value_zscore'].abs() > threshold)

# Display the rows with anomalies
anomalies = final_df[final_df['Anomaly']]

# Print anomalies or handle them as needed
#print(anomalies)

import matplotlib.pyplot as plt

# Set the anomaly detection threshold
#threshold = 1

# Flag anomalies (True for anomalies, False otherwise)
final_df['Anomaly'] = (final_df['Value_zscore'].abs() > threshold)

# Plotting the data points and highlighting anomalies
plt.figure(figsize=(12, 6))
plt.plot(final_df.index, final_df['Value'], label='Data Points')
plt.scatter(final_df[final_df['Anomaly']].index, final_df[final_df['Anomaly']]['Value'], color='red', label='Anomalies', marker='x')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('duration_milliseconds_bucket Z-Score Model')
plt.legend()
plt.show()
print(f"Number of anomalies detected: {len(anomalies)}")



In [ ]:
#IQR

import matplotlib.pyplot as plt

# Step 1: Calculate Q1 (25th percentile) and Q3 (75th percentile) for the 'Value' column
Q1 = final_df['Value'].quantile(0.25)
Q3 = final_df['Value'].quantile(0.75)

# Step 2: Calculate IQR (Interquartile Range)
IQR = Q3 - Q1

# Step 3: Define the lower and upper bounds for anomalies
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 4: Identify anomalies
anomalies = final_df[(final_df['Value'] < lower_bound) | (final_df['Value'] > upper_bound)]

print(f"Number of anomalies detected: {len(anomalies)}")

# Step 5: Plotting the graph without converting 'Time' to datetime
plt.figure(figsize=(10, 6))

# Plot all values
plt.plot(final_df.index, final_df['Value'], label='Value', color='blue')

# Highlight anomalies
plt.scatter(final_df[final_df['Anomaly']].index, final_df[final_df['Anomaly']]['Value'], color='red', label='Anomalies')

# Adding titles and labels
plt.title('duration_milliseconds_bucket IQR Model')
plt.xlabel('index')
plt.ylabel('Value')
plt.legend()

# Show the plot
plt.xticks(rotation=45)  # Rotate x labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()



In [ ]:
#One-Class SVM Model

from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
#file_path = 'C:/Users/abhis/Desktop/CloudBuilders/Grafana-Anomaly/Grafana-Anomaly_Detection/Anomaly_grafana/dataframes/calls_total_df.csv'
#final_df = pd.read_csv(file_path)

# Strip any leading/trailing spaces from column names
final_df.columns = final_df.columns.str.strip()

# Select the numerical feature for One-Class SVM (e.g., 'Value')
X = final_df[['Value']]

# Standardize the feature
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the One-Class SVM model
ocsvm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)  # You can adjust the parameters based on your needs
ocsvm.fit(X_scaled)

# Predict anomalies (1 for normal, -1 for anomaly)
final_df['anomaly'] = ocsvm.predict(X_scaled)

# Display the first few rows with the anomaly column
#final_df[['Value', 'anomaly']].head()

# Convert the 'Time' column to datetime format if not already
# final_df['Time'] = pd.to_datetime(final_df['Time'])

# Plot the values over time
plt.figure(figsize=(12, 6))
plt.plot(final_df.index, final_df['Value'], label='Value', color='blue')

# Highlight anomalies
anomalies = final_df[final_df['anomaly'] == -1]
plt.scatter(anomalies.index, anomalies['Value'], color='red', label='Anomalies')

print(f"Number of anomalies detected: {len(anomalies)}")

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('duration_milliseconds_bucket One-Class SVM Model')
plt.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# LOF Model

from sklearn.neighbors import LocalOutlierFactor
import pandas as pd

# Load the dataset
#df = pd.read_csv('your_dataset.csv')  # Use the correct path to your CSV

# Extract the 'Value' column for LOF analysis
X = final_df[['Value']].values

# Apply Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)  # Adjust parameters as needed
final_df['LOF_Score'] = lof.fit_predict(X)

# Add a column to indicate if a point is an outlier (-1) or not (1)
final_df['Outlier'] = final_df['LOF_Score'] == -1

# Display the dataframe with LOF scores and outlier labels
print(final_df[['Time', 'Value', 'LOF_Score', 'Outlier']].head())

import matplotlib.pyplot as plt

# Plot the data
plt.figure(figsize=(12, 6))
plt.plot(final_df.index, final_df['Value'], label='Value', color='blue',  alpha=0.6)

# Highlight anomalies
anomalies = final_df[final_df['Outlier'] == True]
plt.scatter(anomalies.index, anomalies['Value'], color='red', label='Anomalies')

print(f"Number of anomalies detected: {len(anomalies)}")

# Adding labels and title
plt.title('duration_milliseconds_bucket (LOF) Model')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()

# Show plot
plt.show()

In [ ]:
#ARIMA Model

import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# Step 1: Load the CSV file
# file_path = 'C:/Users/abhis/Desktop/CloudBuilders/Grafana-Anomaly/Grafana-Anomaly_Detection/Anomaly_grafana/dataframes/calls_total_df.csv'
# df = pd.read_csv(file_path)

# Step 2: Parse the 'Time' column as datetime and set it as the index
final_df['Time'] = pd.to_datetime(final_df['Time'])
final_df.set_index('Time', inplace=True)

# Step 3: Resample the data by summing the 'Value' column every 15 minutes to handle any irregular intervals
df_resampled = final_df['Value'].resample('5T').sum()

# Step 4: Split the data into training and testing sets (80% train, 20% test)
train_size = int(len(df_resampled) * 0.8)
train, test = df_resampled[:train_size], df_resampled[train_size:]

# Step 5: Fit the ARIMA model on the training data
# We'll use a basic ARIMA model with (p, d, q) = (1, 1, 1) as a starting point.
model = ARIMA(train, order=(0, 1, 0))
arima_result = model.fit()

# Step 6: Forecast on the test data
forecast = arima_result.forecast(steps=len(test))

# Step 7: Plot the results
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Training Data')
plt.plot(test.index, test, label='Test Data', color='orange')
plt.plot(test.index, forecast, label='ARIMA Forecast', color='green')
plt.legend()
plt.title('duration_milliseconds_bucket ARIMA Forecast')
plt.show()


In [ ]:
#Prophet Model

In [ ]:
#Seasonal Decomposition of Time Series (STL)

import pandas as pd
from statsmodels.tsa.seasonal import STL
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

# Load the dataset
# file_path = 'C:/Users/abhis/Desktop/CloudBuilders/Grafana-Anomaly/Grafana-Anomaly_Detection/Anomaly_grafana/dataframes/calls_total_df.csv'  # Update this path as needed
# df = pd.read_csv(file_path)
df = final_df

# Convert 'Time' column to datetime and set it as the index
df['Time'] = pd.to_datetime(df['Time'])
df.set_index('Time', inplace=True)

# Assuming data points are collected every 15 seconds in a 1-hour period
period = 240  # 4 data points per minute * 60 minutes = 240 data points per hour

# Perform STL decomposition with the specified period
stl = STL(df['Value'], period=period)
result = stl.fit()

# Plot the decomposed components
result.plot()
plt.show()

# Extract residuals from STL decomposition
residuals = result.resid

# 1. Plot the residuals over time
plt.figure(figsize=(10, 6))
plt.plot(residuals, label='Residuals')
plt.title('Residuals Over Time')
plt.legend()
plt.show()

# 2. Histogram and KDE plot of residuals
# plt.figure(figsize=(10, 6))
# sns.histplot(residuals, kde=True, stat="density", linewidth=0)
# plt.title('Histogram and KDE of Residuals')
# plt.show()

# # 3. ACF and PACF plots of residuals
# plt.figure(figsize=(10, 6))
# plot_acf(residuals.dropna(), lags=40)
# plt.title('Autocorrelation Function (ACF) of Residuals')
# plt.show()

# plt.figure(figsize=(10, 6))
# plot_pacf(residuals.dropna(), lags=40)
# plt.title('Partial Autocorrelation Function (PACF) of Residuals')
# plt.show()

# # 4. Perform Augmented Dickey-Fuller test for stationarity
# adf_result = adfuller(residuals.dropna())

# # Extracting and displaying ADF test results
# adf_output = {
#     'ADF Statistic': adf_result[0],
#     'p-value': adf_result[1],
#     'Number of Lags Used': adf_result[2],
#     'Number of Observations Used': adf_result[3]
# }

# adf_output



In [ ]:
#LSTM Model

In [ ]:
#Autoencoder Model